# Song Classification, Part 2

In this lab, we will build our own song classifer using k-nearest neighbors. This is a two-part lab. This is part 2 of the investigation. 

We will build a classifier that guesses whether a song is hip-hop or country, using only the numbers of times words appear in the song's lyrics.  By the end of the project, you should know how to:

1. Clean and organize a dataset used to test a machine learning model
2. Build a k-nearest neighbors classifier
3. Test a classifier on data

**Advice.** It is always easist when we try to develop our solution incrementally. To perform a complicated table manipulation, break it up into steps, perform each step on a different line, give a new name to each result, and check that each intermediate result is what you expect. You can add any additional names or functions you want to the provided cells. 

To get started, load `datascience`, `numpy`, `plots`.

In [1]:
# Run this cell to set up the notebook, but please don't change it.
import numpy as np
import math
from datascience import *

# These lines set up the plotting functionality and formatting.
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter('ignore', UserWarning)


## Overview: Recap

In Part 1, we completed the following tasks:
1. In section 1, we explored the dataset and split the dataset into training data and test data.
2. In section 2, we ran through a guided example of the k-Nearest Neightbors (k-NN) classification algorithm.

In Part 2, we are going to complete the following tasks:
1. Identify some features.
2. Define a classifier function using your features and the training set.
3. Evaluate its performance (the proportion of correct classifications) on the test set.

### Setting up the Lab

In [2]:
lyrics = Table.read_table('lyrics.csv')

training_proportion = 11/16

num_songs = lyrics.num_rows
num_train = int(num_songs * training_proportion)
num_valid = num_songs - num_train

train_lyrics = lyrics.take(np.arange(num_train))
test_lyrics = lyrics.take(np.arange(num_train, num_songs))

def most_common(label, table):
    return table.group(label).sort('count', descending=True).column(label).item(0)

## 1. Features

Now, we're going to extend our classifier from lab 4 to consider more than two features at a time.

Euclidean distance still makes sense with more than two features. For `n` different features, we compute the difference between corresponding feature values for two songs, square each of the `n`  differences, sum up the resulting numbers, and take the square root of the sum.

** Question 1.1 ** <br/>
Write a function to compute the Euclidean distance between two **arrays** of features of *arbitrary* (but equal) length.  Use it to compute the distance between the first song in the training set and the first song in the test set, *using all of the features*.  (Remember that the title, artist, and genre of the songs are not features.)

**Note:** To convert row objects to arrays, use `np.array`. For example, if `t` was a table, `np.array(t.row(0))` converts row 0 of `t` into an array.

In [3]:
def distance(features1, features2):
    """The Euclidean distance between two arrays of feature values."""
    
    distance_first_to_first = []
    feature_length = len(features1) # or len(features2) as they are equal in number of features
    
    for i in np.arange(feature_length):
        distance = np.sqrt(features1[i]**2-features2[i]**2)
        distance_first_to_first = np.append(distance_first_to_first, distance)
        
    distance_first_to_first


### 1.1. Creating your own feature set

Unfortunately, using all of the features has some downsides.  One clear downside is *computational* -- computing Euclidean distances just takes a long time when we have lots of features.  You might have noticed that in the last question!

So we're going to select just 20.  We'd like to choose features that are very *discriminative*. That is, features which lead us to correctly classify as much of the test set as possible.  This process of choosing features that will make a classifier work well is sometimes called *feature selection*, or more broadly *feature engineering*.

** Question 1.1.1 ** <br/>
Look through the list of features (the labels of the `lyrics` table after the first three).  Choose 20 common words that you think might let you distinguish between country and hip-hop songs. Make sure to choose words that are frequent enough that every song contains at least one of them. Don't just choose the 20 most frequent, though... you can do much better.

You might want to come back to this question later to improve your list, once you've seen how to evaluate your classifier.  The first time you answer this question, spend some time looking through the features, but not more than 15 minutes.

In [5]:
# Set my_20_features to an array of 20 features (strings that are column labels)

import random

feature_table = lyrics.drop(np.arange(3)) 
feature_labels = feature_table.labels
#print(feature_labels)

# experiment with only first 20 features and 10 rows for fast processing 
# then change this line later to include 20 common words instead

feature_labels_20 = random.sample(feature_labels, 20)

selected_features = feature_table.select(feature_labels_20)

#selected_table.show(10)

my_20_features = selected_features
#my_20_features

train_20 = train_lyrics.select(my_20_features)
test_20 = test_lyrics.select(my_20_features)

This test makes sure that you have chosen words such that at least one appears in each song. If you can't find words that satisfy this test just through intuition, try writing code to print out the titles of songs that do not contain any words from your list, then look at the words they do contain.

Next, let's classify the first song from our test set using these features.  You can examine the song by running the cells below. Do you think it will be classified correctly?

In [7]:
print("Song:")
test_lyrics.take(0).select('Title', 'Artist', 'Genre').show()
print("Features:")
test_20.take(0).show()

Song:


Title,Artist,Genre
That Kind of Love,Alison Krauss,Country


Features:


hang,chorus,bathroom,brake,sinner,vieux,holler,sin,thrill,café,golp,thang,marri,inform,então,drop,arena,gefühl,vega,sleev
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


As before, we want to look for the songs in the training set that are most alike our test song.  We will calculate the Euclidean distances from the test song (using the 20 selected features) to all songs in the training set.  You could do this with a `for` loop, but to make it computationally faster, we have provided a function, `fast_distances`, to do this for you.  Read its documentation to make sure you understand what it does.  (You don't need to read the code in its body unless you want to.)

In [8]:
# Just run this cell to define fast_distances.

def fast_distances(test_row, train_rows):
    """An array of the distances between test_row and each row in train_rows.

    Takes 2 arguments:
      test_row: A row of a table containing features of one
        test song (e.g., test_20.row(0)).
      train_rows: A table of features (for example, the whole
        table train_20)."""
    
    assert train_rows.num_columns < 50, "Make sure you're not using all the features of the lyrics table."
    counts_matrix = np.asmatrix(train_rows.columns).transpose()
    diff = np.tile(np.array(test_row), [counts_matrix.shape[0], 1]) - counts_matrix
    distances = np.squeeze(np.asarray(np.sqrt(np.square(diff).sum(1))))
    return distances

** Question 1.1.2 ** <br/>
Use the `fast_distances` function provided above to compute the distance from the first song in the test set to all the songs in the training set, **using your set of 20 features**.  Make a new table called `genre_and_distances` with one row for each song in the training set and two columns:
* The `"Genre"` of the training song
* The `"Distance"` from the first song in the test set 

Ensure that `genre_and_distances` is **sorted in increasing order by distance to the first test song**.

In [9]:
# The staff solution took 4 lines of code.
test_row1 = test_20.row(0)
train_rows1 = train_20

genres = train_lyrics.column('Genre')
distances = fast_distances(test_row1, train_rows1)

genre_and_distances = Table().with_columns("Genre", genres, "Distance", distances).sort(distances, descending=False)
genre_and_distances

Genre,Distance
Hip-hop,0
Country,0
Country,0
Country,0
Country,0
Hip-hop,0
Country,0
Country,0
Country,0
Country,0


** Question 1.1.3 ** <br/>
Now compute the 5-nearest neighbors classification of the first song in the test set.  That is, decide on its genre by finding the most common genre among its 5 nearest neighbors, according to the distances you've calculated.  Then check whether your classifier chose the right genre.  (Depending on the features you chose, your classifier might not get this song right, and that's okay.)

In [11]:
# Set my_assigned_genre to the most common genre among these.

my_assigned_genre = genre_and_distances.take(np.arange(5)).group("Genre").sort("count", descending=True).item(0).column(0).item(0)
# print(my_assigned_genre)

# Set my_assigned_genre_was_correct to True if my_assigned_genre
# matches the actual genre of the first song in the test set.
if (test_lyrics.column("Genre").item(0) == my_assigned_genre):
    my_assigned_genre_was_correct = True
else:
    my_assigned_genre_was_correct = False

print("The assigned genre, {}, was{}correct.".format(my_assigned_genre, " " if my_assigned_genre_was_correct else " not "))

The assigned genre, Country, was correct.


### 1.2. A classifier function

Now we can write a single function that encapsulates the whole process of classification.

** Question 1.2.1 ** <br/>
Write a function called `classify`.  It should take the following four arguments:
* A row of features for a song to classify (e.g., `test_20.row(0)`).
* A table with a column for each feature (e.g., `train_20`).
* An array of classes that has as many items as the previous table has rows, and in the same order.
* `k`, the number of neighbors to use in classification.

It should return the class a `k`-nearest neighbor classifier picks for the given row of features (the string `'Country'` or the string `'Hip-hop'`).

In [13]:
def classify(test_row, train_rows, train_classes, k):
    """Return the most common class among k nearest neigbors to test_row."""
    distances = fast_distances(test_row, train_rows)
    
    
    # an array of training classes with matching number of rows with training rows
    #genres = train_lyrics.column('Genre')
    
    genre_and_distances = Table().with_columns("Genre", train_classes, "Distance", distances).sort(distances, descending=False)
    # print(genre_and_distances)
    predicted_genre = genre_and_distances.take(np.arange(k)).group("Genre").sort("count", descending=True).item(0).column(0).item(0)
    return (predicted_genre)

#train_classes = train_lyrics.column('Genre')
#classify(test_row1, train_rows1, train_classes, 5)

** Question 1.2.2 ** <br/>
Assign `grandpa_genre` to the genre predicted by your classifier for the song "Grandpa Got Runned Over By A John Deere" in the test set, using **9 neighbors** and using your 20 features.

In [15]:
# The staff solution first defined a row object called grandpa_features.

grandpa_features = test_lyrics.where("Title", are.equal_to("Grandpa Got Runned Over By A John Deere")).drop(np.arange(3)).select(my_20_features).row(0)
#grandpa_features.num_columns

train_rows = train_20
#train_20.num_columns

classes = train_lyrics.column('Genre')
#train_classes

k_neighbors = 9
grandpa_genre = classify(grandpa_features, train_rows, classes, k_neighbors)
grandpa_genre

'Country'

In [16]:
grade("tests/q1_2_2.py")

Finally, when we evaluate our classifier, it will be useful to have a classification function that is specialized to use a fixed training set and a fixed value of `k`.

** Question 1.2.3 ** <br/>
Create a classification function that takes as its argument a row containing your 20 features and classifies that row using the 5-nearest neighbors algorithm with `train_20` as its training set.

In [17]:
def classify_one_argument(row):
    k_neighbors = 5
    training_set = train_20 
    classes = train_lyrics.column('Genre')
    song_class = classify(row, training_set, classes, k_neighbors)
    return song_class

# When you're done, this should produce 'Hip-hop' or 'Country'.
classify_one_argument(test_20.row(0))

'Country'

### 1.3. Evaluating your classifier

Now that it's easy to use the classifier, let's see how accurate it is on the whole test set.

**Question 1.3.1.** Use `classify_one_argument` and `apply` to classify every song in the test set.  Name these guesses `test_guesses`.  **Then**, compute the proportion of correct classifications. 

In [19]:
test_features = test_lyrics.drop(np.arange(3)).select(my_20_features)
#test_rows.show(5)
#test_rows.labels

# magic apply function normally takes column(s) as an argument but we need to feed each row instead
test_guesses = Table().with_column("rows", test_features.rows).apply(classify_one_argument, "rows")
#test_guesses

guess_table = test_lyrics.select(np.arange(3)).with_column("Test Guesses", test_guesses)
#guess_table.show(10)

#compare two columns side by side to see if they match and put them on a new table
correct_guesses = guess_table.where("Genre", are.equal_to, "Test Guesses")
#correct_guesses.num_rows
#test_lyrics.num_rows

proportion_correct = correct_guesses.num_rows/test_lyrics.num_rows
proportion_correct

0.6301115241635687

In [1]:
# Further Enhancing Our Algorithm:
# the following is my resampling experiment in selecting 20 best features to maximze accuracy
# experiment with random 20 features and resample 10000 times 
# then change this line later to include 20 common words instead
# yields 78% overall accuracy

In [ ]:

import random

feature_table = lyrics.drop(np.arange(3)) 
feature_labels = feature_table.labels
#print(feature_labels)


repititions = 10000
sample_features = []
sample_proportions = []

for i in range(repititions):
    feature_labels_20 = random.sample(feature_labels, 20)
    
    sample_features = np.append(sample_features, str(feature_labels_20))
    
    selected_features = feature_table.select(feature_labels_20)

#selected_table.show(10)

    my_20_features = selected_features

    train_20 = train_lyrics.select(my_20_features)
    test_20 = test_lyrics.select(my_20_features)

    # test_features = test_lyrics.select(my_20_features)

    test_guesses = Table().with_column("rows", test_20.rows).apply(classify_one_argument, "rows")

    guess_table = test_lyrics.select(np.arange(3)).with_column("Test Guesses", test_guesses)
#guess_table.show(10)

#compare two columns side by side to see if they match and put them on a new table

    correct_guesses = guess_table.where("Genre", are.equal_to, "Test Guesses")

#correct_guesses.num_rows

    proportion_correct = correct_guesses.num_rows/test_20.num_rows
    sample_proportions = np.append(sample_proportions, proportion_correct)

#Table().with_columns("features", sample_features)
#Table().with_columns("proportions", sample_proportions)
top_ten_predictions = Table().with_columns("features", sample_features, 
                                           "proportions", sample_proportions).sort(sample_proportions, descending=True)

top_ten_predictions.show(10)
top_20_features = top_ten_predictions.row(0).item(0)
print (top_20_features)


At this point, you've gone through one cycle of classifier design.  Let's summarize the steps:
1. From available data, select test and training sets.
2. Choose an algorithm you're going to use for classification.
3. Identify some features.
4. Define a classifier function using your features and the training set.
5. Evaluate its performance (the proportion of correct classifications) on the test set.

Congratulations! You've successfully built a song classifier!

Disclaimer: Please note that this is a part of the student project assignment from Data Science Foundations Course offered by University of California at Berkeley and shown here for demonstration purposes only. Only the codes and solutions are mine. (Anjani K Shiwakoti, March 20, 2019).